### **1. Import Library**
# Mengimpor pustaka yang diperlukan


In [61]:
import pandas as pd
import re
from IPython.display import display
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

### **2. Load Dataset**
# Membaca dataset training dan testing dari CSV

In [85]:
training_path = "../data/raw/data_training.csv"
testing_path = "../data/raw/data_testing.csv"

df_train = pd.read_csv(training_path, sep=';')
df_test = pd.read_csv(testing_path, sep=';')

def display_table(df, title):
    styled_df = df.head(3).style.set_properties(**{"text-align": "left"}).set_caption(title)
    display(styled_df)

### **2. Fungsi case folding**
# Memproses teks dengan mengubah huruf kapital menjadi huruf kecil

In [86]:
def case_folding(text):
    return text.lower()

df_train["case_folding"] = df_train["hadis"].apply(case_folding)
display_table(df_train[["hadis", "case_folding"]], "📌 Case Folding - Data Training")

,hadis,case_folding
0,"Dari Muhammad, hamba Allah dan Rasul-Nya untuk Heraclius. Penguasa Romawi, Keselamatan bagi siapa yang mengikuti petunjuk. Kemudian daripada itu, aku mengajakmu dengan seruan Islam; masuk Islamlah kamu, maka kamu akan selamat, Allah akan memberi pahala kepadamu dua kali. Namun jika kamu berpaling, maka kamu menanggung dosa rakyat kamu, dan: Hai ahli kitab, marilah (berpegang) kepada suatu kalimat (ketetapan) yang tidak ada perselisihan antara kami dan kamu, bahwa tidak kita sembah kecuali Allah dan tidak kita persekutukan dia dengan sesuatupun dan tidak (pula) sebagian kita menjadikan sebagian yang lain sebagai Rabb selain Allah. Jika mereka berpaling, maka katakanlah kepada mereka: Saksikanlah, bahwa kami adalah orang-orang yang berserah diri (kepada Allah). Abu Sufyan menuturkan: Setelah Heraclius menyampaikan apa yang dikatakannya dan selesai membaca surat tersebut, terjadilah hiruk pikuk dan suara-suara ribut, sehingga mengusir kami. Aku berkata kepada teman-temanku setelah kami diusir keluar; sungguh dia telah diajak kepada urusan Anak Abu Kabsyah. Heraclius mengkhawatirkan kerajaan Romawi.Pada masa itupun aku juga khawatir bahwa Muhammad akan berjaya, sampai akhirnya (perasaan itu hilang setelah) Allah memasukkan aku ke dalam Islam. Dan adalah Ibnu An Nazhur, seorang Pembesar Iliya' dan Heraclius adalah seorang uskup agama Nashrani, dia menceritakan bahwa pada suatu hari ketika Heraclius mengunjungi Iliya' dia sangat gelisah, berkata sebagian komandan perangnya: Sungguh kami mengingkari keadaanmu. Selanjutnya kata Ibnu Nazhhur, Heraclius adalah seorang ahli nujum yang selalu memperhatikan perjalanan bintang-bintang. Dia pernah menjawab pertanyaan para pendeta yang bertanya kepadanya; Pada suatu malam ketika saya mengamati perjalanan bintang-bintang, saya melihat raja Khitan telah lahir, siapakah di antara ummat ini yang di khitan? Jawab para pendeta; Yang berkhitan hanyalah orang-orang Yahudi, janganlah anda risau karena orang-orang Yahudi itu. Perintahkan saja keseluruh negeri dalam kerajaan anda, supaya orang-orang Yahudi di negeri tersebut di bunuh. Ketika itu di hadapakan kepada Heraclius seorang utusan raja Bani Ghasssan untuk menceritakan perihal Rasulullah , setelah orang itu selesai bercerita, lalu Heraclius memerintahkan agar dia diperiksa, apakah dia berkhitan ataukah tidak. Seusai di periksa, ternyata memang dia berkhitam. Lalu di beritahukan orang kepada Heraclius. Heraclius bertanya kepada orang tersebut tentang orang-orang Arab yang lainnya, di khitankah mereka ataukah tidak? Dia menjawab; Orang Arab itu di khitan semuanya. Heraclius berkata; 'inilah raja ummat, sesungguhnya dia telah terlahir. Kemudian heraclisu berkirim surat kepada seorang sahabatnya di Roma yang ilmunya setarf dengan Heraclisu (untuk menceritakan perihal kelahiran Nabi Muhammad ). Sementara itu, ia meneruskan perjalanannya ke negeri Himsha, tetapi sebelum tiba di Himsha, balasan surat dari sahabatnya itu telah tiba terlebih dahulu. Sahabatnya itu menyetujui pendapat Heraclius bahwa Muhammad telah lahir dan bahwa beliau memang seorang Nabi. Heraclius lalu mengundang para pembesar Roma supaya datang ke tempatnya di Himsha, setelah semuanya hadir dalam majlisnya, Heraclius memerintahkan supaya mengunci semua pintu. Kemudian dia berkata; 'Wahai bangsa rum, maukah anda semua beroleh kemenangan dan kemajuan yang gilang gemilang, sedangkan kerajaan tetap utuh di tangan kita? Kalau mau, akuilah Muhammad sebagai Nabi!. Mendengar ucapan itu, mereka lari bagaikan keledai liar, padahal semua pintu telah terkunci. Melihat keadaan yang demikian, Heraclius jadi putus harapan yang mereka akan beriman (percaya kepada kenabian Muhammad). Lalu di perintahkannya semuanya untuk kembali ke tempatnya masing-masing seraya berkata; Sesungguhnya saya mengucapkan perkataan saya tadi hanyalah sekedar menguji keteguhan hati anda semua. Kini saya telah melihat keteguhan itu. Lalu mereka sujud di hadapan Heraclius dan mereka senang kepadanya. D

### **3. Fungsi noise removal**
# Menghilangkan karakter yang tidak diperlukan seperti tanda baca, angka, dan karakter khusus lainnya

In [87]:
def remove_noise(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Hanya menyisakan huruf dan spasi
    return text

df_train["remove_noise"] = df_train["case_folding"].apply(remove_noise)
display_table(df_train[["case_folding", "remove_noise"]], "📌 Remove Noise - Data Training")

,case_folding,remove_noise
0,"dari muhammad, hamba allah dan rasul-nya untuk heraclius. penguasa romawi, keselamatan bagi siapa yang mengikuti petunjuk. kemudian daripada itu, aku mengajakmu dengan seruan islam; masuk islamlah kamu, maka kamu akan selamat, allah akan memberi pahala kepadamu dua kali. namun jika kamu berpaling, maka kamu menanggung dosa rakyat kamu, dan: hai ahli kitab, marilah (berpegang) kepada suatu kalimat (ketetapan) yang tidak ada perselisihan antara kami dan kamu, bahwa tidak kita sembah kecuali allah dan tidak kita persekutukan dia dengan sesuatupun dan tidak (pula) sebagian kita menjadikan sebagian yang lain sebagai rabb selain allah. jika mereka berpaling, maka katakanlah kepada mereka: saksikanlah, bahwa kami adalah orang-orang yang berserah diri (kepada allah). abu sufyan menuturkan: setelah heraclius menyampaikan apa yang dikatakannya dan selesai membaca surat tersebut, terjadilah hiruk pikuk dan suara-suara ribut, sehingga mengusir kami. aku berkata kepada teman-temanku setelah kami diusir keluar; sungguh dia telah diajak kepada urusan anak abu kabsyah. heraclius mengkhawatirkan kerajaan romawi.pada masa itupun aku juga khawatir bahwa muhammad akan berjaya, sampai akhirnya (perasaan itu hilang setelah) allah memasukkan aku ke dalam islam. dan adalah ibnu an nazhur, seorang pembesar iliya' dan heraclius adalah seorang uskup agama nashrani, dia menceritakan bahwa pada suatu hari ketika heraclius mengunjungi iliya' dia sangat gelisah, berkata sebagian komandan perangnya: sungguh kami mengingkari keadaanmu. selanjutnya kata ibnu nazhhur, heraclius adalah seorang ahli nujum yang selalu memperhatikan perjalanan bintang-bintang. dia pernah menjawab pertanyaan para pendeta yang bertanya kepadanya; pada suatu malam ketika saya mengamati perjalanan bintang-bintang, saya melihat raja khitan telah lahir, siapakah di antara ummat ini yang di khitan? jawab para pendeta; yang berkhitan hanyalah orang-orang yahudi, janganlah anda risau karena orang-orang yahudi itu. perintahkan saja keseluruh negeri dalam kerajaan anda, supaya orang-orang yahudi di negeri tersebut di bunuh. ketika itu di hadapakan kepada heraclius seorang utusan raja bani ghasssan untuk menceritakan perihal rasulullah , setelah orang itu selesai bercerita, lalu heraclius memerintahkan agar dia diperiksa, apakah dia berkhitan ataukah tidak. seusai di periksa, ternyata memang dia berkhitam. lalu di beritahukan orang kepada heraclius. heraclius bertanya kepada orang tersebut tentang orang-orang arab yang lainnya, di khitankah mereka ataukah tidak? dia menjawab; orang arab itu di khitan semuanya. heraclius berkata; 'inilah raja ummat, sesungguhnya dia telah terlahir. kemudian heraclisu berkirim surat kepada seorang sahabatnya di roma yang ilmunya setarf dengan heraclisu (untuk menceritakan perihal kelahiran nabi muhammad ). sementara itu, ia meneruskan perjalanannya ke negeri himsha, tetapi sebelum tiba di himsha, balasan surat dari sahabatnya itu telah tiba terlebih dahulu. sahabatnya itu menyetujui pendapat heraclius bahwa muhammad telah lahir dan bahwa beliau memang seorang nabi. heraclius lalu mengundang para pembesar roma supaya datang ke tempatnya di himsha, setelah semuanya hadir dalam majlisnya, heraclius memerintahkan supaya mengunci semua pintu. kemudian dia berkata; 'wahai bangsa rum, maukah anda semua beroleh kemenangan dan kemajuan yang gilang gemilang, sedangkan kerajaan tetap utuh di tangan kita? kalau mau, akuilah muhammad sebagai nabi!. mendengar ucapan itu, mereka lari bagaikan keledai liar, padahal semua pintu telah terkunci. melihat keadaan yang demikian, heraclius jadi putus harapan yang mereka akan beriman (percaya kepada kenabian muhammad). lalu di perintahkannya semuanya untuk kembali ke tempatnya masing-masing seraya berkata; sesungguhnya saya mengucapkan perkataan saya tadi hanyalah sekedar menguji keteguhan hati anda semua. kini saya telah melihat keteguhan itu. lalu mereka sujud di hadapan heraclius dan mereka senang kepad

### **4. Fungsi tokenization**
# Memecah kalimat menjadi kata-kata

In [88]:
def tokenize(text):
    return text.split()

df_train["tokenizing"] = df_train["remove_noise"].apply(tokenize)
display_table(df_train[["remove_noise", "tokenizing"]], "📌 Tokenizing - Data Training")

### **5. Fungsi stopword removal**
# Menghilangkan kata-kata yang tidak memiliki makna seperti kata hubung, kata depan, dan lain-lain

In [89]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def remove_stopwords(tokens):
    return [word for word in tokens if stopword.remove(word) != '']

df_train["remove_stopwords"] = df_train["tokenizing"].apply(remove_stopwords)
display_table(df_train[["tokenizing", "remove_stopwords"]], "📌 Remove Stopwords - Data Training")


# def remove_stopwords_text(words):
#     return [word for word in words if word not in stopword.remove(' '.join(words)).split()]

# df_train["stopword_removal_text"] = df_train["tokenizing"].apply(remove_stopwords_text)
# display_table(df_train[["tokenizing", "stopword_removal_text"]], "📌 Stopword Removal - Data Training")


### **6. Fungsi stemming**
# Mengubah kata-kata yang tidak baku menjadi kata baku

In [75]:
from tqdm import tqdm
tqdm.pandas()  

factori = StemmerFactory()
stemmer = factori.create_stemmer()

# 🔹 Fungsi untuk stemming setiap kata dalam daftar token
def stem_text(tokens):
    return [stemmer.stem(word) for word in tokens]


df_train["stemming"] = df_train["remove_stopwords"].progress_apply(stem_text)
display_table(df_train[["remove_stopwords", "stemming"]], "📌 Stemming - Data Training")

  0%|          | 0/5601 [00:00<?, ?it/s]

100%|██████████| 5601/5601 [31:37<00:00,  2.95it/s]  


,remove_stopwords,stemming
0,"['muhammad', 'hamba', 'allah', 'rasulnya', 'heraclius', 'penguasa', 'romawi', 'keselamatan', 'siapa', 'mengikuti', 'petunjuk', 'kemudian', 'aku', 'mengajakmu', 'seruan', 'islam', 'masuk', 'islamlah', 'kamu', 'kamu', 'selamat', 'allah', 'memberi', 'pahala', 'kepadamu', 'kali', 'kamu', 'berpaling', 'kamu', 'menanggung', 'dosa', 'rakyat', 'kamu', 'hai', 'ahli', 'kitab', 'marilah', 'berpegang', 'suatu', 'kalimat', 'ketetapan', 'perselisihan', 'kamu', 'sembah', 'allah', 'persekutukan', 'sesuatupun', 'sebagian', 'menjadikan', 'sebagian', 'rabb', 'allah', 'berpaling', 'katakanlah', 'saksikanlah', 'orangorang', 'berserah', 'diri', 'allah', 'abu', 'sufyan', 'menuturkan', 'heraclius', 'menyampaikan', 'apa', 'dikatakannya', 'selesai', 'membaca', 'surat', 'tersebut', 'terjadilah', 'hiruk', 'pikuk', 'suarasuara', 'ribut', 'mengusir', 'aku', 'berkata', 'temantemanku', 'diusir', 'keluar', 'sungguh', 'diajak', 'urusan', 'anak', 'abu', 'kabsyah', 'heraclius', 'mengkhawatirkan', 'kerajaan', 'romawipada', 'masa', 'itupun', 'aku', 'khawatir', 'muhammad', 'berjaya', 'akhirnya', 'perasaan', 'hilang', 'allah', 'memasukkan', 'aku', 'islam', 'ibnu', 'an', 'nazhur', 'seorang', 'pembesar', 'iliya', 'heraclius', 'seorang', 'uskup', 'agama', 'nashrani', 'menceritakan', 'suatu', 'hari', 'heraclius', 'mengunjungi', 'iliya', 'sangat', 'gelisah', 'berkata', 'sebagian', 'komandan', 'perangnya', 'sungguh', 'mengingkari', 'keadaanmu', 'selanjutnya', 'kata', 'ibnu', 'nazhhur', 'heraclius', 'seorang', 'ahli', 'nujum', 'selalu', 'memperhatikan', 'perjalanan', 'bintangbintang', 'pernah', 'menjawab', 'pertanyaan', 'pendeta', 'bertanya', 'kepadanya', 'suatu', 'malam', 'mengamati', 'perjalanan', 'bintangbintang', 'melihat', 'raja', 'khitan', 'lahir', 'siapakah', 'ummat', 'khitan', 'jawab', 'pendeta', 'berkhitan', 'hanyalah', 'orangorang', 'yahudi', 'janganlah', 'risau', 'orangorang', 'yahudi', 'perintahkan', 'keseluruh', 'negeri', 'kerajaan', 'orangorang', 'yahudi', 'negeri', 'tersebut', 'bunuh', 'hadapakan', 'heraclius', 'seorang', 'utusan', 'raja', 'bani', 'ghasssan', 'menceritakan', 'perihal', 'rasulullah', 'orang', 'selesai', 'bercerita', 'lalu', 'heraclius', 'memerintahkan', 'diperiksa', 'berkhitan', 'ataukah', 'seusai', 'periksa', 'ternyata', 'memang', 'berkhitam', 'lalu', 'beritahukan', 'orang', 'heraclius', 'heraclius', 'bertanya', 'orang', 'tersebut', 'orangorang', 'arab', 'lainnya', 'khitankah', 'ataukah', 'menjawab', 'orang', 'arab', 'khitan', 'semuanya', 'heraclius', 'berkata', 'inilah', 'raja', 'ummat', 'sesungguhnya', 'terlahir', 'kemudian', 'heraclisu', 'berkirim', 'surat', 'seorang', 'sahabatnya', 'roma', 'ilmunya', 'setarf', 'heraclisu', 'menceritakan', 'perihal', 'kelahiran', 'nabi', 'muhammad', 'meneruskan', 'perjalanannya', 'negeri', 'himsha', 'tiba', 'himsha', 'balasan', 'surat', 'sahabatnya', 'tiba', 'terlebih', 'sahabatnya', 'menyetujui', 'pendapat', 'heraclius', 'muhammad', 'lahir', 'beliau', 'memang', 'seorang', 'nabi', 'heraclius', 'lalu', 'mengundang', 'pembesar', 'roma', 'datang', 'tempatnya', 'himsha', 'semuanya', 'hadir', 'majlisnya', 'heraclius', 'memerintahkan', 'mengunci', 'semua', 'pintu', 'kemudian', 'berkata', 'wahai', 'bangsa', 'rum', 'maukah', 'semua', 'beroleh', 'kemenangan', 'kemajuan', 'gilang', 'gemilang', 'kerajaan', 'tetap', 'utuh', 'tangan', 'kalau', 'mau', 'akuilah', 'muhammad', 'nabi', 'mendengar', 'ucapan', 'lari', 'bagaikan', 'keledai', 'liar', 'padahal', 'semua', 'pintu', 'terkunci', 'melihat', 'keadaan', 'heraclius', 'jadi', 'putus', 'harapan', 'beriman', 'percaya', 'kenabian', 'muhammad', 'lalu', 'perintahkannya', 'semuanya', 'tempatnya', 'masingmasing', 'berkata', 'sesungguhnya', 'mengucapkan', 'perkataan', 'tadi', 'hanyalah', 'sekedar', 'menguji', 'keteguhan', 'hati', 'semua', 'kini', 'melihat', 'keteguhan', 'lalu', 'sujud', 'hadapan', 'heraclius', 'senang', 'kepadanya', 'demikianlah', 'akhir', 'kisah', 'heraclius', 'riwayatkan', 'shalih', 'bin', 'kaisan', 'yunus', 'mamar', 'az', 'zuhri

### **7. Fungsi preprocessing**
# Menggabungkan semua fungsi preprocessing yang telah dibuat sebelumnya

In [92]:
df_test["case_folding"] = df_test["hadis"].apply(case_folding)
df_test["remove_noise"] = df_test["case_folding"].apply(remove_noise)
df_test["tokenizing"] = df_test["remove_noise"].apply(tokenize)
df_test["remove_stopwords"] = df_test["tokenizing"].apply(remove_stopwords)
df_test["stemming"] = df_test["remove_stopwords"].apply(stem_text)

### **8. Simpan hasil preprocessing**


In [95]:
processed_training_path = "../data/processed/preprocessed_training.csv"
processed_testing_path = "../data/processed/preprocessed_testing.csv"

df_train[["stemming", "anjuran", "larangan", "informasi"]].to_csv(processed_training_path, sep=';', index=False)
df_test[["stemming", "anjuran", "larangan", "informasi"]].to_csv(processed_testing_path, sep=';', index=False)
# df_test["hadis"] = df_test["hadis"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
# df_test = df_test.rename(columns={"hadis": "stemming"})
# df_test.to_csv(processed_testing_path, sep=';', index=False)


print(f"Preprocessing selesai! Data training tersimpan di {processed_training_path}")
print(f"Preprocessing selesai! Data testing tersimpan di {processed_testing_path}")

KeyError: "['stemming'] not in index"